In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

from argus import Model


class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

    
class Net2(nn.Module):
    def __init__(self, n_classes, p_dropout=0.5):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=p_dropout)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, n_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [ ]:
class TestModel1(Model):
    nn_module = Net2
    optimizer = torch.optim.SGD
    loss = torch.nn.CrossEntropyLoss


params = {
    'nn_module': {'n_classes': 10, 'p_dropout': 0.1},
    'optimizer': {'lr': 0.01},
    'device': 'cpu'
}


model = TestModel1(params)
model.__dict__

In [ ]:
class TestModel2(Model):
    nn_module = Net1
    optimizer = 'SGD'
    loss = 'NLLLoss'


params = {
    'optimizer': {'lr': 0.01},
}


model = TestModel2(params)
model.__dict__

In [ ]:
class TestModel3(Model):
    nn_module = Net1
    device = torch.device('cpu')

params = {
    'optimizer': ('SGD', {'lr': 0.01, 'momentum': 0.5}),
    'loss': 'NLLLoss',
}


model = TestModel3(params)
model.__dict__

In [ ]:
class TestModel4(Model):
    nn_module = {
        'net1': Net1,
        'net2': Net2
    }
    optimizer = {
        'sgd': 'SGD',
        'adam': torch.optim.Adam
    }
    loss = {
        'cross_entropy': nn.CrossEntropyLoss,
        'nll': 'NLLLoss'
    }


params = {
    'nn_module': ('net2', {'n_classes': 10, 'p_dropout': 0.1}),
    'optimizer': ('adam', {'lr': 0.01}),
    'loss': 'cross_entropy',
    'device': 'cpu'
}

model = TestModel4(params)
model.__dict__

In [ ]:
try:
    from cnn_finetune import make_model
except:
    raise ImportError('cnn_finetune not installed')

    
class CnnFinetune(Model):
    nn_module = make_model


params = {
    'nn_module': {'model_name': 'resnet18', 'num_classes': 10, 'pretrained': False},
    'optimizer': ('Adam', {'lr': 0.01}),
    'loss': 'CrossEntropyLoss',
    'device': 'cpu'
}

model = CnnFinetune(params)
model.__dict__